In [1]:
import csv
import json
from pprint import pprint
import csv
import os
import json
import spotipy

from SPARQLWrapper import SPARQLWrapper, TURTLE, JSON
from pandas_ml import ConfusionMatrix
import pandas as pd
from sklearn import metrics


with open('../data/poi_artist_path.json','r') as input_fp:
    poi_artist_path = json.load(input_fp)
poi_artist_path

/usr/local/lib/python2.7/site-packages/ipython-5.3.0-py2.7.egg/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


{u'http://dbpedia.org/resource/Allianz_Riviera': {u'http://dbpedia.org/resource/Christophe_Marchand': [u'http://dbpedia.org/resource/Christophe_Marchand',
   u'http://purl.org/dc/terms/subject',
   u'http://dbpedia.org/resource/Category:French_swimmers',
   u'http://www.w3.org/2004/02/skos/core#broader',
   u'http://dbpedia.org/resource/Category:Swimming_in_France',
   u'http://www.w3.org/2004/02/skos/core#broader',
   u'http://dbpedia.org/resource/Category:Sport_in_France_by_sport',
   u'http://www.w3.org/2004/02/skos/core#broader',
   u'http://dbpedia.org/resource/Category:Football_in_France',
   u'http://www.w3.org/2004/02/skos/core#broader',
   u'http://dbpedia.org/resource/Category:Football_venues_in_France',
   u'http://purl.org/dc/terms/subject',
   u'http://dbpedia.org/resource/Allianz_Riviera',
   u'0.724614183300277',
   u'1.4529508355825853'],
  u'http://dbpedia.org/resource/Marcel_Duchamp': [u'http://dbpedia.org/resource/Marcel_Duchamp',
   u'http://purl.org/dc/terms/subjec

In [2]:
def get_artist_tracks(artist_name):
    sp = spotipy.Spotify()
    sp.trace = False

    # get matching artist

    results = sp.search(q='artist:' + artist_name, type='artist')
    artists = results['artists']['items']
    print(artists)
    artist = None
    if len(artists) == 0:
        return []
    artist = max(artists, key=lambda x: x['popularity'])

    top_tracks = sp.artist_top_tracks(artist['id'], country='FR')['tracks']
    return_tracks = []
    for track in top_tracks:
        return_tracks.append((track['id'], track['name']))
    if len(return_tracks) < 10:
        return_tracks = []
    return return_tracks


def get_dbpedia_label(uri):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
                        SELECT ?label ?preferredLabel 
            WHERE { 
               <%s> rdfs:label ?label .
            FILTER (lang(?label) = "" || lang(?label) = "en") 
               OPTIONAL { 
                 <%s> rdfs:label ?preferredLabel . 
                 FILTER (lang(?preferredLabel) = "" || lang(?preferredLabel) = "tk") 
               } 
            } limit 1
                        """ % (uri, uri))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    label = None
    for artist in results['results']['bindings']:
        try:
            label = artist['preferredLabel']['value']
        except KeyError:
            label = artist['label']['value']
        return label
    
def get_dbpedia_name(uri):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery("""
                        SELECT ?name
            WHERE { 
               <%s> dbo:name ?name .
            } limit 1
                        """ % uri)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    label = None
    for artist in results['results']['bindings']:
            label = artist['name']['value']
    return label

In [3]:
for poi in poi_artist_path:
    for artist in poi_artist_path[poi]:
        path = poi_artist_path[poi][artist] 
        poi_artist_path[poi][artist] = dict()
        poi_artist_path[poi][artist]['path'] = path
        poi_artist_path[poi][artist]['tracks'] = []

In [4]:
for poi in poi_artist_path:
    for artist in poi_artist_path[poi]:
        print(artist)

http://dbpedia.org/resource/Sergey_Dmitriyev
http://dbpedia.org/resource/Marcel_Duchamp
http://dbpedia.org/resource/Richard_I_of_England
http://dbpedia.org/resource/Christophe_Marchand
http://dbpedia.org/resource/Yannick_Noah
http://dbpedia.org/resource/Philippe_Manoury
http://dbpedia.org/resource/Cali_(singer)
http://dbpedia.org/resource/Richard_I_of_England
http://dbpedia.org/resource/Marc_Blitzstein
http://dbpedia.org/resource/Philippe_Lavil
http://dbpedia.org/resource/Marcel_Duchamp
http://dbpedia.org/resource/Yannick_Noah
http://dbpedia.org/resource/Christophe_Marchand
http://dbpedia.org/resource/Pablo_de_Sarasate
http://dbpedia.org/resource/Patrick_Doyle
http://dbpedia.org/resource/Marcel_Duchamp
http://dbpedia.org/resource/Guy_Béart
http://dbpedia.org/resource/Jean-Louis_Aubert
http://dbpedia.org/resource/Patrick_Bruel
http://dbpedia.org/resource/Jean_Ferrat
http://dbpedia.org/resource/Daniel_Balavoine
http://dbpedia.org/resource/Charles_Aznavour
http://dbpedia.org/resource/Jacq

In [5]:
uri = 'http://dbpedia.org/resource/John_Milton_(composer)'
get_dbpedia_name(uri)

In [6]:
get_artist_tracks('yannick noah')

SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/search?q=artist%3Ayannick+noah&type=artist&limit=10&offset=0:
 No token provided